# Spracovanie a analýza dát galaxií s využitím hlbokého učenia

## Príprava pracovného prostredia pre prácu s YOLO

Inštalácia YOLO

In [ ]:
!pip install opencv-contrib-python
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

## Vytvorenie YAML súboru
# Vstupy
train: cesta k textovému súboru trénovacích dát \
val: cesta k textovému súboru validačných dát \
nc: počet tried, ktoré model bude rozlišovať \
names: názvy jednotlivých tried zaradom

Príklad YAML súboru \
train: /home/jovyan/data/lightning/JulianaGazdova/trainbach.txt \
val: /home/jovyan/data/lightning/JulianaGazdova/valbach.txt \
nc: 1 \
names: ['Edge-on galaxy']

### Trénovanie modelu

cesta pre spustenie trénovania: yolov5/train.py \
cesta k yaml súboru vstupných dát: galaxies/galaxies.yaml \
cesta k yaml súboru modelu yolov5s: yolov5/models/yolov5s.yaml

In [ ]:
!python yolov5/train.py --img 1280 --rect --batch 8 --epochs 50 --data galaxies/galaxies.yaml --weights yolov5/models/yolov5s.yaml --cache --save-period 2 --workers 0

### Detekcia modelu

cesta pre spustenie detegovania: yolov5/detect.py \
cesta k yaml súboru vstupných dát: /home/jovyan/data/lightning/JulianaGazdova/yolo/testovacia \
cesta k yaml súboru modelu yolov5s: yolov5/runs/train/exp50/weights/best.pt 

In [ ]:
!python yolov5/detect.py --source /home/jovyan/data/lightning/JulianaGazdova/yolo/testovacia --weights yolov5/runs/train/exp51/weights/best.pt --iou 0.25 --conf 0.36 --img 1280 --save-txt --save-conf

### Validácia modelu

cesta pre spustenie validácie: yolov5/val.py \
cesta k yaml súboru vstupných dát:/home/jovyan/data/lightning/JulianaGazdova/yolo/galaxies/galaxies.yaml \
cesta k yaml súboru modelu yolov5s:  yolov5/runs/train/exp51/weights/best.pt


In [ ]:
!python yolov5/val.py --data /home/jovyan/data/lightning/JulianaGazdova/yolo/galaxies/galaxies.yaml --weights yolov5/runs/train/exp51/weights/best.pt --iou 0.3 --conf 0.24 --img 1280 --save-txt --save-conf --workers 0

# Príprava pracovného prostredia pre prácu s SCSS-Net

Inštalácia SCSS-Net

In [ ]:
!git clone https://github.com/space-lab-sk/scss-net.git

!pip install -U albumentations
!pip install mega.py
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

Inštalácia knižníc

In [ ]:
import sys, os
!pip install -r requirements.txt
sys.path.append('../scss-net/src')


Importovanie knižníc 

In [ ]:
from mega import Mega
import zipfile
import glob
from datetime import datetime
from PIL import Image
import matplotlib.pylab as plt
from matplotlib.gridspec import GridSpec
from sklearn.model_selection import train_test_split
import albumentations
from ImageDataAugmentor.image_data_augmentor import *
from tensorflow.keras.callbacks import ModelCheckpoint

from model_scss_net import scss_net
from metrics import dice_np, iou_np, dice, iou
from utils import plot_imgs, plot_metrics

from PIL import Image, ImageEnhance

Nastavenie veľkosti obrázka, batch size a epóch a pomenovanie modelu

In [ ]:
IMG_SIZE = 64  # resize imgs to 256x256
BATCH_SIZE = 20 # set batch size
SEED = 20       # set seed for reproducibility
EPOCHS = 1000    # Set number of epochs

MODEL_NAME = "model_galaxie_vsetky_1000_ep_drop_6" # Specify model name
model_filename = f"{MODEL_NAME}.h5"                # Specify path where to save model

Príprava dát

In [ ]:
imgs = glob.glob("data/all_galaxies/*.jpg")
masks = glob.glob("data/all_masks/*.jpg")

print(f"Imgs number = {len(imgs)}\nMasks number = {len(masks)}")

imgs_list = []
masks_list = []
for image, mask in zip(imgs, masks):
    #ig = Image.open(image)
    #enhancer = ImageEnhance.Color(ig)
    #enhancer.enhance(25) toto namiesto Image.open(image)
    imgs_list.append(np.array(Image.open(image).convert("L").resize((IMG_SIZE, IMG_SIZE))))
    masks_list.append(np.array(Image.open(mask).convert("L").resize((IMG_SIZE, IMG_SIZE))))
    
    
x = np.asarray(imgs_list, dtype=np.float32)/255
y = np.asarray(masks_list, dtype=np.float32)/255

# Reshape to (n_imgs, height, width, channels)
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
y = y.reshape(y.shape[0], y.shape[1], y.shape[2], 1)

Rozdelenie množín

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=123, shuffle=True)

Vykreslenie obrázkov

In [ ]:
plot_imgs(imgs=x, masks=y, n_imgs=5).show()

### Trénovanie modelu SCSS

In [ ]:
# Load model architecture with optimal parameteres
model = scss_net( 
    input_shape,
    filters=32,       
    layers=4,
    batch_norm=True,
    drop_prob=0.6)

# Compile model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",  
    metrics=[iou, dice])

# Set steps parameters acording to size of training set and size of batch
STEPS = x_train.shape[0] // BATCH_SIZE        

# Set Callback that saves only best weights
callback_checkpoint = ModelCheckpoint(
    model_filename,
    verbose=1,
    monitor="val_loss",
    save_best_only=True)

# Train model
history = model.fit(
    x_train,
    y_train,
    steps_per_epoch=STEPS,
    epochs=EPOCHS,
    validation_data=(x_val, y_val),
    callbacks=[callback_checkpoint],
    verbose=2)

# # Plot training history (Metrics and Loss)
plot_metrics(history).show()


Uloženie modelu

In [ ]:
model.save_weights(model_filename)

plot_imgs(imgs=x_val, masks=y_val, predictions=y_pred, n_imgs=5).show()

Predikovanie masiek

In [ ]:
imgs_test = glob.glob("data/test_galaxies/*.jpg")
masks_test = glob.glob("data/test_masks/*.jpg")

print(f"Imgs number = {len(imgs_test)}\nMasks number = {len(masks_test)}")

# Load data and convert imgs to np.array
imgs_test_list = []
masks_test_list = []
for image, mask in zip(imgs_test, masks_test):
    imgs_test_list.append(np.array(Image.open(image).convert("L").resize((IMG_SIZE, IMG_SIZE))))
    masks_test_list.append(np.array(Image.open(mask).convert("L").resize((IMG_SIZE, IMG_SIZE))))

# Normalization from (0; 255) to (0; 1)
x_test = np.asarray(imgs_test_list, dtype=np.float32)/255
y_test = np.asarray(masks_test_list, dtype=np.float32)/255

# Reshape to (n_imgs, height, width, channels)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], y_test.shape[2], 1)

y_pred = model.predict(x_test)  
plot_imgs(imgs=x_test, masks=y_test, predictions=y_pred, n_imgs=20).show()

Vyhodnotenie metrík

In [ ]:
y_pred_bin = np.where(y_pred > 0.83, 1, 0)  # Binarize predicted values

dice = np.round(dice_np(y_test, y_pred), 4)
iou_test = np.round(iou_np(y_test, y_pred), 4)

dice_tresh = np.round(dice_np(y_test, y_pred_bin), 4)
iou_test_tresh = np.round(iou_np(y_test, y_pred_bin), 4)

print(f"Test:\nDice: {dice} Dice_tresh: {dice_tresh}\n IoU: {iou_test} IoU_tresh: {iou_test_tresh}\n")